# Prevendo séries temporais - números de produção elétrica

Vamos entender como funciona o algortimo de LSTM com o exemplo de séries temporais.

### Importando as bibliotecas necessárias:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import pandas as pd
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error

Função para receber os dados e as divisões na entrada X e na saída Y, dividindo em `n` dias passados como entrada X. 

In [ ]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

### Carregando o dataset:

In [ ]:
np.random.seed(7)

#Carrega o dataset
dataframe = read_csv('https://telescopeinstorage.blob.core.windows.net/datasets/Eletrical_Production.csv', index_col = 'DATE', engine='python')
dataframe.head(150)

In [ ]:
dataframe.tail(150)

In [ ]:
dataframe.plot(figsize=(10,10))

### Transformando os dados

Coletando dados de produção apartir de 1980 até o último dia de registro de produção.

In [ ]:
dataframeFilter =  dataframe.loc['1980-01-01':]

In [ ]:
dataframeFilter.head()

In [ ]:
dataframeFilter.tail()

In [ ]:
dataframeFilter.plot(figsize=(10,10))

In [ ]:
X = dataframeFilter[['IPG2211A2N']].values
X = X.astype('float32')

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(X)

### Normalizando os dados:

In [ ]:
#Normaliza o dataset
scaler = MinMaxScaler(feature_range=(0, 1))

#scaler = StandardScaler()
X = scaler.fit_transform(X)

#O StandardScaler é um tipo de scaler que dimensiona os valores mínimo e máximo para 0 e 1, respectivamente. 
#Enquanto o MinMaxscaler dimensiona todos os valores entre min e max para que fiquem dentro de um intervalo de min a max. 

In [ ]:
X

### Separando os dados de treino e teste:

In [ ]:
train_size = int(len(X) * 0.7)
test_size = len(X) - train_size

train = X[0:train_size,:]
test =  X[train_size:len(X),:]

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
plt.plot(train)

In [ ]:
plt.plot(test)

### Definindo o número de dias para predição.

Vamos utilizar nesse caso a análise com base nos 3 últimos dias.

In [ ]:
look_back = 3 # quantidade de dias passados

X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)


In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_train.shape

In [ ]:
# reshape da entrada para ser [amostras, time steps, caracteristicas]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
X_train[1]

In [ ]:
y_train[1]

In [ ]:
X_train.shape

In [ ]:
# Cria e treina a rede LSTM
model = Sequential()
model.add(LSTM(8, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mae']) #Mean absolute error

model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1)

In [ ]:
# Predições
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test) #novos dados

In [ ]:
# Faz a transformação inversa das predições
trainPredict = scaler.inverse_transform(trainPredict)
y_train = scaler.inverse_transform([y_train])

testPredict = scaler.inverse_transform(testPredict)
y_test = scaler.inverse_transform([y_test])

In [ ]:
# Calcula o RMSE
trainScore = math.sqrt(mean_squared_error(y_train[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# Prepara o vetor para plotagem do conjunto de treinamento
trainPredictPlot = np.empty_like(X)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back-1:len(trainPredict)+look_back-1, :] = trainPredict

In [ ]:
# Prepara o vetor para plotagem do conjunto das predições de teste
testPredictPlot = np.empty_like(X)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2):len(X)-2, :] = testPredict

In [ ]:
# Plota o dataset, as predições sobre o conjunto de treinamento e teste
plt.figure(figsize=(10,8))
plt.plot(scaler.inverse_transform(X))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
# Plota o dataset, as predições sobre o conjunto de treinamento e teste
plt.figure(figsize=(10,8))
plt.plot(scaler.inverse_transform(X))
plt.plot(trainPredictPlot)
#plt.plot(testPredictPlot)
plt.show()

In [ ]:
# Plota o dataset, as predições sobre o conjunto de treinamento e teste
plt.figure(figsize=(10,8))
plt.plot(scaler.inverse_transform(X))
#plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
Dados_Teste = np.array(list(y_test)) #nosso target
Dados_Preditos = np.array(testPredict) #nossas predições

Resultado_Predicao = pd.DataFrame({'Actual': Dados_Teste.flatten(), 'Predicted': Dados_Preditos.flatten()})
Resultado_Predicao

In [ ]:
y_test

In [ ]:
testPredict

In [ ]:
#Dados_Preditos

In [ ]:
df_validacao= Resultado_Predicao.head(50)
df_validacao.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
Resultado_Predicao.to_csv('Resultado_Predicao.csv', sep=',')